# OptimizedBot optimization

In [2]:
import arcade
import random
import sys

sys.path.append('..')
from blockade import Blockade
from players.OptimizedBot import OptimizedBot

In [3]:
# test series of games with different random seeds
for seed in range(3):
    random.seed(seed)
    game = Blockade(player1=OptimizedBot(weights=[1.0, 1.0, 1.0, 0.0], verbose=False),
                    player2=OptimizedBot(weights=[0.0, 0.0, 0.0, 0.0], verbose=False),
                    arena_size=20,
                    tile_size=50,
                    game_speed=999,
                    mute_sound=True,
                    window_hidden=True,
                    verbose=False)
    game.setup()
    game.run()
    print(game.outcome)
    arcade.close_window()
    del game

1
1
1
